<a href="https://colab.research.google.com/github/Talhaahmd/Personality-Evaluation-Model/blob/main/Questionnaire_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

In [2]:
train = pd.read_csv("/content/Train_data.csv")
train

,Submission Date,First Name,Last Name,E-mail,How do you feel about trying new experiences or activities?,Do you often seek out novel or unconventional ways of solving problems?,Are you someone who sets goals and works diligently to achieve them?,"Do you feel energized by social interactions, or do you prefer spending time alone?",Do you tend to speak up and assert yourself in group settings?,"How do you typically respond to stress or adversity? Are you easily rattled, or do you remain calm under pressure?",How anxious do you get as soon as your deadlines start to approach?,What is the single biggest motivating factor in your life?,Do you often procrastinate?,Any final comments?
0,14-May-24,Ahmad,Isfar,f2021-171@bnu.edu.pk,5,5,6,0,3,6,10,4,1,no
1,14-May-24,Amna,Mazhar,s2023-157@bnu.edu.pk,10,8,7,0,9,8,10,1,1,I tend to overcome all of these problems overtime
2,14-May-24,Waleed,Nadeem,waleednadeem158@gmail.com,9,7,6,1,5,3,6,2,1,..
3,14-May-24,Hamza Ali,Shahid,f2019-278@bnu.edu.pk,10,9,8,0,5,2,5,5,1,No comment
4,14-May-24,Muhammad,Asad,f2021-348@bnu.edu.pk,10,5,5,1,6,1,6,3,1,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,14-May-24,Muhammad Mohsin Saeed,Saeed Ahmad,mohsinsaeed356@gmail.com,8,5,7,1,7,8,1,5,1,NaN
97,14-May-24,Zoha,Ajmal,f2021-172@bnu.edu.pk,9,8,6,0,3,1,7,5,0,NaN
98,14-May-24,Tallat,Nasim,tallat.asif@gmail.com,2,10,10,1,8,4,2,1,1,NaN
99,14-May-24,Ebrahim,Arshad,f2021-230@bnu.edu.pk,5,10,10,0,1,1,10,5,0,NaN


In [3]:
train = train.rename(columns={'How do you feel about trying new experiences or activities?':'Openness1'}, inplace=False)
train = train.rename(columns={'Do you often seek out novel or unconventional ways of solving problems?':'Openness2'}, inplace=False)
train = train.rename(columns={'Are you someone who sets goals and works diligently to achieve them?':'Conscientiousness1'}, inplace=False)
train = train.rename(columns={'Do you feel energized by social interactions, or do you prefer spending time alone?':'Extraversion1'}, inplace=False)
train = train.rename(columns={'Do you tend to speak up and assert yourself in group settings?':'Extraversion2'}, inplace=False)
train = train.rename(columns={'How do you typically respond to stress or adversity? Are you easily rattled, or do you remain calm under pressure?':'Neuroticism1'}, inplace=False)
train = train.rename(columns={'How anxious do you get as soon as your deadlines start to approach?':'Neuroticism2'}, inplace=False)
train = train.rename(columns={'What is the single biggest motivating factor in your life?':'Condition1'}, inplace=False)
train = train.rename(columns={'Do you often procrastinate?':'NeuroConsciousness'}, inplace=False)

In [4]:
train = train.drop(columns=['Any final comments?'], inplace=False)

In [5]:
train['Openness'] = (train['Openness1'] + train['Openness2']) / 2
train['Extraversion'] = (train['Extraversion1'] + train['Extraversion2']) / 2
train['Neuroticism'] = (train['Neuroticism1'] + train['Neuroticism2']) / 2
train['Conscientiousness'] = (train['Conscientiousness1']) / 2

In [6]:
def adjust_columns(row):
    if row['Condition1'] == 1:
        row['Conscientiousness'] += 3
    elif row['Condition1'] == 2:
        row['Conscientiousness'] += 3
        row['Openness'] -= 2
        row['Extraversion'] += 3
        row['Neuroticism'] += 3
    elif row['Condition1'] == 3:
        row['Conscientiousness'] += 3
        row['Openness'] += 3
        row['Extraversion'] -= 2
    elif row['Condition1'] == 4:
        row['Conscientiousness'] += 3
        row['Openness'] += 3
    elif row['Condition1'] == 5:
        row['Conscientiousness'] += 3
        row['Openness'] += 3
    return row


In [7]:
def map_to_category(value):
    if value <= 4:
        return "Very Low"
    elif 4 < value <= 7:
        return "Low"
    elif 7 < value <= 11:
        return "Mid"
    elif 11 < value <= 14:
        return "High"
    else:
        return "Very High"

In [8]:
train = train.apply(adjust_columns, axis=1)

In [9]:
for trait in ['Openness', 'Extraversion', 'Neuroticism', 'Conscientiousness']:
    train[f'{trait}_Category'] = train[trait].apply(map_to_category)

In [10]:
train

,Submission Date,First Name,Last Name,E-mail,Openness1,Openness2,Conscientiousness1,Extraversion1,Extraversion2,Neuroticism1,...,Condition1,NeuroConsciousness,Openness,Extraversion,Neuroticism,Conscientiousness,Openness_Category,Extraversion_Category,Neuroticism_Category,Conscientiousness_Category
0,14-May-24,Ahmad,Isfar,f2021-171@bnu.edu.pk,5,5,6,0,3,6,...,4,1,8.0,1.5,8.0,6.0,Mid,Very Low,Mid,Low
1,14-May-24,Amna,Mazhar,s2023-157@bnu.edu.pk,10,8,7,0,9,8,...,1,1,9.0,4.5,9.0,6.5,Mid,Low,Mid,Low
2,14-May-24,Waleed,Nadeem,waleednadeem158@gmail.com,9,7,6,1,5,3,...,2,1,6.0,6.0,7.5,6.0,Low,Low,Mid,Low
3,14-May-24,Hamza Ali,Shahid,f2019-278@bnu.edu.pk,10,9,8,0,5,2,...,5,1,12.5,2.5,3.5,7.0,High,Very Low,Very Low,Low
4,14-May-24,Muhammad,Asad,f2021-348@bnu.edu.pk,10,5,5,1,6,1,...,3,1,10.5,1.5,3.5,5.5,Mid,Very Low,Very Low,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,14-May-24,Muhammad Mohsin Saeed,Saeed Ahmad,mohsinsaeed356@gmail.com,8,5,7,1,7,8,...,5,1,9.5,4.0,4.5,6.5,Mid,Very Low,Low,Low
97,14-May-24,Zoha,Ajmal,f2021-172@bnu.edu.pk,9,8,6,0,3,1,...,5,0,11.5,1.5,4.0,6.0,High,Very Low,Very Low,Low
98,14-May-24,Tallat,Nasim,tallat.asif@gmail.com,2,10,10,1,8,4,...,1,1,6.0,4.5,3.0,8.0,Low,Low,Very Low,Mid
99,14-May-24,Ebrahim,Arshad,f2021-230@bnu.edu.pk,5,10,10,0,1,1,...,5,0,10.5,0.5,5.5,8.0,Mid,Very Low,Low,Mid


In [11]:
features = [
    'Openness1', 'Openness2',
    'Conscientiousness1',
    'Extraversion1', 'Extraversion2',
    'Neuroticism1', 'Neuroticism2',
    'Condition1', 'NeuroConsciousness'
]
labels = ['Openness', 'Extraversion', 'Neuroticism', 'Conscientiousness']


In [12]:
X = train[features]
y = train[labels]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [14]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
r2 = r2_score(y_test, y_pred, multioutput='raw_values')
for i, trait in enumerate(labels):
    print(f'{trait} - Mean Squared Error: {mse[i]}, R-squared: {r2[i]}')

Openness - Mean Squared Error: 0.7386272727272728, R-squared: 0.8816239735099337
Extraversion - Mean Squared Error: 1.8156386363636359, R-squared: 0.5085184004474274
Neuroticism - Mean Squared Error: 1.8604068181818183, R-squared: 0.817577613452188
Conscientiousness - Mean Squared Error: 0.20807272727272721, R-squared: 0.6687263157894737


In [21]:
for i, trait in enumerate(labels):
    print(f'Predicted {trait} values:')
    print(y_pred[:, i])

Predicted Openness values:
[ 8.855  9.35   4.71   6.675 11.38   4.155  6.78   9.04   5.54   7.795
  8.36 ]
Predicted Extraversion values:
[2.26  4.855 5.14  5.875 4.11  4.965 4.56  3.14  4.595 3.065 4.28 ]
Predicted Neuroticism values:
[ 3.72   8.69  11.445  6.905  7.56   8.685  4.795  4.675  4.925  7.06
  7.385]
Predicted Conscientiousness values:
[6.305 7.26  5.335 6.98  7.32  5.8   6.47  6.22  7.065 6.18  7.085]


In [15]:
test_data = pd.DataFrame({
    'Openness1': [8], 'Openness2': [6],
    'Conscientiousness1': [9],
    'Extraversion1': [0], 'Extraversion2': [8],
    'Neuroticism1': [3], 'Neuroticism2': [1],
    'Condition1': [4], 'NeuroConsciousness': [1]
})

predicted_traits = model.predict(test_data)
predicted_data = pd.DataFrame(predicted_traits, columns=labels)
print(predicted_data)

   Openness  Extraversion  Neuroticism  Conscientiousness
0      10.1         2.645        3.885               6.73


In [16]:
name_search = train[train[''] == '']

email_search = train[train['E-mail'] == '']

In [ ]:
name_search

,Submission Date,First Name,Last Name,E-mail,Openness1,Openness2,Conscientiousness1,Extraversion1,Extraversion2,Neuroticism1,...,Condition1,NeuroConsciousness,Openness,Extraversion,Neuroticism,Conscientiousness,Openness_Category,Extraversion_Category,Neuroticism_Category,Conscientiousness_Category
17,14-May-24,Mohsin,Sabir,f2021-277@bnu.edu.pk,10,8,8,0,8,5,...,1,0,9.0,4.0,6.0,7.0,Mid,Very Low,Low,Low


In [ ]:
email_search

,Submission Date,First Name,Last Name,E-mail,Openness1,Openness2,Conscientiousness1,Extraversion1,Extraversion2,Neuroticism1,...,Condition1,NeuroConsciousness,Openness,Extraversion,Neuroticism,Conscientiousness,Openness_Category,Extraversion_Category,Neuroticism_Category,Conscientiousness_Category
